# AI image detection
## Installing the required dependecies and importing

In [1]:
%pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip list

Package                           Version
--------------------------------- -------------------
absl-py                           2.3.1
aiobotocore                       2.19.0
aiohappyeyeballs                  2.4.4
aiohttp                           3.11.10
aioitertools                      0.7.1
aiosignal                         1.2.0
alabaster                         0.7.16
altair                            5.5.0
anaconda-anon-usage               0.7.1
anaconda-auth                     0.8.6
anaconda-catalogs                 0.2.0
anaconda-cli-base                 0.5.2
anaconda-client                   1.13.0
anaconda-navigator                2.6.6
anaconda-project                  0.11.1
annotated-types                   0.6.0
anyio                             4.7.0
appdirs                           1.4.4
apyori                            1.1.2
archspec                          0.2.3
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
arrow           

In [3]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.utils.data import Subset

## Data manipulation
Here we start off by loading in data from a csv file. Then we create an ImageDataset class to handle getting the images and subsequently manipualte those images to make training less cost expensive in terms of compute time. We also create the training and test sets.

Load data and define the ImageDataset

In [4]:

train_df = pd.read_csv("dataset/train.csv")
test_df  = pd.read_csv("dataset/test_v2_labels.csv")
test_df=test_df.rename(columns={"id": "file_name"})
train_df.head()
print(train_df['label'].unique())
print(train_df['label'].dtype)


[1 0]
int64


In [5]:
class ImageDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform #save the dataframe, imagedirectory and transform

    def __len__(self):
        return len(self.df) #rows in the df

    def __getitem__(self, idx):
        row = self.df.iloc[idx] #takes one row from the df and makes the image path for it, open the image and convert to rgb
        img_path = os.path.join(self.img_dir, row["file_name"])
        image = Image.open(img_path).convert("RGB")
        label = torch.tensor(row["label"], dtype=torch.float) #creates a label and changes it to a pytorch tensor

        if self.transform:
            image = self.transform(image) #use the transform

        return image, label

Manipulating images

In [6]:
transform = transforms.Compose([
    transforms.Resize((256, 256)), #resize image to 256x256px
    transforms.RandomHorizontalFlip(p=0.5), # random right or left flip for generalization
    transforms.RandomRotation(10), #turn of up to 10 degrees
    #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # image generalization for model robustness
    transforms.ToTensor(), #turns the image into a pytorch tensor and normalizes pixels from 0-1
])

Creating train and test sets

In [7]:
train_dataset = ImageDataset(train_df, "dataset", transform) #load the training dataset,
test_dataset  = ImageDataset(test_df,  "dataset", transform)

# Take 20% of the training dataset
subset_size = int(0.1 * len(train_dataset))
train_subset = Subset(train_dataset, range(subset_size))

# Take 20% of the test dataset
subset_size_test = int(0.1 * len(test_dataset))
test_subset = Subset(test_dataset, range(subset_size_test))


train_loader = DataLoader(train_subset, batch_size=32, shuffle=True) #for training take a batch of 32 images at once for faster training, and shuffle the images each epoch for generalization
test_loader  = DataLoader(test_subset,  batch_size=32, shuffle=False) #For testing shuffle is turned off to get repetable results


Image shape: torch.Size([3, 256, 256])
Label: tensor(0.)


In [8]:
image, label = train_dataset[10001]   
print("Image shape:", image.shape)
print("Label:", label)
plt.imshow(image.permute(1, 2, 0))
plt.title(f"Label: {label.item()}")
plt.axis("off")

plt.show()

## Creating a Pytorch CNN model

Device configuration

In [9]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Hyperparameters

In [10]:
num_epochs = 2
learing_rate= 0.0005

Defining the Pytroch model<br>
Layers consist of conuvlution+relu and pooling for feature learning after which classification

In [11]:
class AiImageClassifier(nn.Module):
    def __init__ (self):
        super().__init__()

        self.conv_block1=nn.Sequential(
            nn.Conv2d(in_channels=3, #3 input channels rgb
                      out_channels=32, # 32 feature maps
                      kernel_size=3, # 3x3 patch of the image
                      padding=1), #padding so outputs the same as input
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv_block2=nn.Sequential(
            nn.Conv2d(32,64,3,padding=1),   # looks for patterns
            nn.ReLU(),                      # makes the model non linear
            nn.MaxPool2d(2)                 # Shrinks the image, keeping strong signals
        )
        self.conv_block3=nn.Sequential(
            nn.Conv2d(64,128,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1=nn.Linear(128*32*32,256) # 128x32x32 features into 256
        self.fc2=nn.Linear(256,1) # single layer for binary classificiation
        self.sigmoid=nn.Sigmoid() # 0 or 1
        

    
    def forward(self, x):
        x=self.conv_block1(x)   # [batch, 32, 128, 128]
        x=self.conv_block2(x)   # [batch, 64, 64, 64]
        x=self.conv_block3(x)   # [batch, 128, 32, 32]
        x=x.view(x.size(0),-1)  # turns 3d feature map into 1D vector
        x=self.fc1(x)           
        x=self.fc2(x)           # combines layers for final prediciton
        return self.sigmoid(x) # returns 0 or 1

Training the model

In [12]:
model=AiImageClassifier().to(device)
criterion=nn.BCELoss() # Binary Cross Entropy 
optimizer=optim.Adam(model.parameters(),lr=learing_rate) 

n_total_steps=len(train_loader) 
for epoch in range(num_epochs): 
    model.train() 
    running_loss=0 # track loss 
    
    # Looping over all batches in the training set 
    for i, (images,labels) in enumerate(train_loader): 
        images=images.to(device)
        labels=labels.float().to(device) 
        
        #forward pass and computing loss 
        outputs=model(images).squeeze(1) 
        loss=criterion(outputs,labels)
        
        #backward and optimize 
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 
        
        #update weights 
        running_loss+=loss.item()
        #print avg loss every 50 steps 
        if (i+1)%50==0: 
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}') 
    print(f'Epoch [{epoch+1}/{num_epochs}] finished, Average Loss: {running_loss / n_total_steps:.4f}') 
print('Finished training')

Epoch [1/2], Step [50/250], Loss: 0.2081
Epoch [1/2], Step [100/250], Loss: 0.3103
Epoch [1/2], Step [150/250], Loss: 0.4959
Epoch [1/2], Step [200/250], Loss: 0.2498
Epoch [1/2], Step [250/250], Loss: 0.1696
Epoch [1/2] finished, Average Loss: 0.4521
Epoch [2/2], Step [50/250], Loss: 0.1205
Epoch [2/2], Step [100/250], Loss: 0.1424
Epoch [2/2], Step [150/250], Loss: 0.2686
Epoch [2/2], Step [200/250], Loss: 0.1792
Epoch [2/2], Step [250/250], Loss: 0.6722
Epoch [2/2] finished, Average Loss: 0.2943
Finished training


Calculating accuracy

In [13]:
model.eval()
n_correct=0
n_samples=0

#no gradient computation
with torch.no_grad():
    for images,labels in test_loader:
        images=images.to(device)
        labels=labels.float().to(device)
        
        outputs=model(images).squeeze()
        preds=(outputs > 0.5).int() #Probabilities to binary predicitons
        
        #Compare preds and count correct ones
        n_correct+= (preds==labels.int()).sum().item() 
        n_samples+=labels.size(0)

acc=100*n_correct/n_samples
print(f'Accuracy of the network: {acc}%')

Accuracy of the network: 56.498194945848375%
